PyCity Schools Analysis
Dennis O'Leary
Homework 4

The most telling data is observed when groupig the average scores and passing rates by school type, per-student budget, and school size(total students).

Schools with higher per-student budgets did not yeild higher test scores.  More analysis is required as to the reason for this.  It is possible that students with special needs would require a higher budget and would not necessarily score higher on standardized tests.

Small and medium size schools produced similar test scores and large schools (over 2000 students) were significantly lower.

Charter schools test scores were significantly higher than district schools.  This is to be expected as many times charter schools are able to cherry-pick the best students from the district.


In [159]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [160]:
# calculates the totals
totSch = len(school_data['school_name'].unique())
totStu = len(student_data['student_name'])
totBud = school_data['budget'].sum()
totMath = student_data['math_score'].mean()
totRead = student_data['reading_score'].mean()
# uses bins to passes/fails, counts them and calculates percentages
bins = [0, 69, 100]
grpNames = ['fail', 'pass']
readRate = student_data.loc[:, ['student_name', 'reading_score']]
mathRate = student_data.loc[:, ['student_name', 'math_score']]
readRate["reading_score"] = pd.cut(readRate["reading_score"], bins, labels=grpNames)
mathRate["math_score"] = pd.cut(mathRate["math_score"], bins, labels=grpNames)
readRate = readRate.groupby('reading_score')
mathRate = mathRate.groupby('math_score')
readRate = readRate.count()
mathRate = mathRate.count()
readPass = (readRate.iat[1, 0]/readRate['student_name'].sum())*100
mathPass = (mathRate.iat[1, 0]/mathRate['student_name'].sum())*100
overPass = (mathPass+readPass)/2

# create a dataframe to display the output
distSum = {'Total Schools': [totSch],
          'Total Students': [totStu],
          'Total Budget': [totBud],
          'Average Math Score': [totMath],
          'Average Reading Score': [totRead],
          'Passing Math': [mathPass],
          'Passing Reading': [readPass],
          'Overall Passing Rate': [overPass]}
distSum = pd.DataFrame(distSum)
# format columns
distSum['Total Budget'] = distSum['Total Budget'].map("${:.2f}".format)
distSum['Average Math Score'] = distSum['Average Math Score'].map("{:.2f}".format)
distSum['Average Reading Score'] = distSum['Average Reading Score'].map("{:.2f}".format)
distSum['Passing Math'] = distSum['Passing Math'].map("{:.2f}%".format)
distSum['Passing Reading'] = distSum['Passing Reading'].map("{:.2f}%".format)
distSum['Overall Passing Rate'] = distSum['Overall Passing Rate'].map("{:.2f}%".format)

distSum

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
0,15,39170,$24649428.00,78.99,81.88,74.98%,85.81%,80.39%


In [208]:
# creates a dataframe grouped by school and type using .mean() which gives average scores and the budget and total students
topSchools = school_data_complete.loc[:, ['school_name', 'type', 'size', 'budget', 'math_score', 'reading_score']]
topSchools = topSchools.rename(columns={'school_name': 'School Name', 'type': 'School Type'})
topSchGrp = topSchools.groupby(['School Name', 'School Type'])
topSchGrp = topSchGrp.mean()
# adds a column for the Per Student Budget
topSchGrp['Per Student Budget'] = topSchGrp['budget']/topSchGrp['size']
# uses bins to passes/fails for math and reading
bins = [0, 69, 100]
grpNames = ['fail', 'pass']
readRate = school_data_complete.loc[:, ['school_name', 'student_name', 'reading_score']]
mathRate = school_data_complete.loc[:, ['school_name', 'student_name', 'math_score']]
readRate["reading_score"] = pd.cut(readRate["reading_score"], bins, labels=grpNames)
mathRate["math_score"] = pd.cut(mathRate["math_score"], bins, labels=grpNames)
readRate = readRate.groupby(['school_name','reading_score'])
mathRate = mathRate.groupby(['school_name','math_score'])
readRate = readRate.count()
mathRate = mathRate.count()
mathPass = []
mathFail = []
readPass = []
readFail = []
i = 1
# loops through math and reading pass/fail bins for each school 
while i < 30:
    mathPass.append(mathRate.iat[i, 0])
    mathFail.append(mathRate.iat[i-1, 0])
    readPass.append(readRate.iat[i, 0])
    readFail.append(readRate.iat[i-1, 0])
    i = i + 2
# calculates math pass% for each school and adds it as a column in our dataframe
topSchGrp['% Passing Math'] = mathPass
topSchGrp['mathFail'] = mathFail
topSchGrp['% Passing Math'] = (topSchGrp['% Passing Math']/(topSchGrp['% Passing Math']+topSchGrp['mathFail']))*100
del topSchGrp['mathFail']
# calculates reading pass% for each school and adds it as a column in our dataframe
topSchGrp['% Passing Reading'] = readPass
topSchGrp['readFail'] = readFail
topSchGrp['% Passing Reading'] = (topSchGrp['% Passing Reading']/(topSchGrp['% Passing Reading']+topSchGrp['readFail']))*100
del topSchGrp['readFail']
# calculates overall pass% for each school and adds it as a column in our dataframe
topSchGrp['% Overall Passing Rate'] = (topSchGrp['% Passing Math']+topSchGrp['% Passing Reading'])/2
# renames, sorts, and formats our columns
topSchGrp = topSchGrp.rename(columns={'size': 'Total Students',
                                        'budget': 'Total School Budget',
                                        'math_score': 'Average Math Score',
                                        'reading_score': 'Average Reading Score'})
topSchGrp = topSchGrp.loc[:,['Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score',
                            'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']] 
topSchGrp = topSchGrp.sort_values(by=['% Overall Passing Rate'], ascending=False)
botSchGrp = topSchGrp.sort_values(by=['% Overall Passing Rate'], ascending=True)
# creates unformated dataframe with indexes reset for later use
unformSch = topSchGrp.reset_index()
topSchGrp['Total Students'] = topSchGrp['Total Students'].map("{:.0f}".format)
topSchGrp['Total School Budget'] = topSchGrp['Total School Budget'].map("${:.2f}".format)
topSchGrp['Average Math Score'] = topSchGrp['Average Math Score'].map("{:.2f}".format)
topSchGrp['Average Reading Score'] = topSchGrp['Average Reading Score'].map("{:.2f}".format)    
topSchGrp['% Passing Math'] = topSchGrp['% Passing Math'].map("{:.2f}%".format)
topSchGrp['% Passing Reading'] = topSchGrp['% Passing Reading'].map("{:.2f}%".format)
topSchGrp['% Overall Passing Rate'] = topSchGrp['% Overall Passing Rate'].map("{:.2f}%".format)
topSchGrp['Per Student Budget'] = topSchGrp['Per Student Budget'].map("${:.2f}".format)

topSchGrp.head()

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,School Type,,,,,,,,
Cabrera High School,Charter,1858,$1081356.00,$582.00,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,1635,$1043130.00,$638.00,83.42,83.85,93.27%,97.31%,95.29%
Pena High School,Charter,962,$585858.00,$609.00,83.84,84.04,94.59%,95.95%,95.27%
Griffin High School,Charter,1468,$917500.00,$625.00,83.35,83.82,93.39%,97.14%,95.27%
Wilson High School,Charter,2283,$1319574.00,$578.00,83.27,83.99,93.87%,96.54%,95.20%


In [209]:
botSchGrp['Total Students'] = botSchGrp['Total Students'].map("{:.0f}".format)
botSchGrp['Per Student Budget'] = botSchGrp['Per Student Budget'].map("${:.2f}".format)
botSchGrp['Total School Budget'] = botSchGrp['Total School Budget'].map("${:.2f}".format)
botSchGrp['Average Math Score'] = botSchGrp['Average Math Score'].map("{:.2f}".format)
botSchGrp['Average Reading Score'] = botSchGrp['Average Reading Score'].map("{:.2f}".format)    
botSchGrp['% Passing Math'] = botSchGrp['% Passing Math'].map("{:.2f}%".format)
botSchGrp['% Passing Reading'] = botSchGrp['% Passing Reading'].map("{:.2f}%".format)
botSchGrp['% Overall Passing Rate'] = botSchGrp['% Overall Passing Rate'].map("{:.2f}%".format)
botSchGrp.head()

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,School Type,,,,,,,,
Rodriguez High School,District,3999,$2547363.00,$637.00,76.84,80.74,66.37%,80.22%,73.29%
Figueroa High School,District,2949,$1884411.00,$639.00,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,District,2917,$1910635.00,$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,4761,$3094650.00,$650.00,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,District,2739,$1763916.00,$644.00,77.10,80.75,68.31%,79.30%,73.80%


In [210]:
# creates dataframe and groups by School Name and grade and finds the average score for each grade in each school
math = school_data_complete.loc[:, ['school_name', 'grade', 'math_score']]
mathGrp = math.groupby(['school_name', 'grade'])
mathGrp = mathGrp.mean()
# loops through grouped dataframe and creates lists of each grades averages
nine = []
ten = []
eleven = []
twelve = []
i = 3
while i < 60:
    nine.append(mathGrp.iat[i,0])
    i = i + 4
i = 0
while i < 60:
    ten.append(mathGrp.iat[i,0])
    i = i + 4
i = 1
while i < 60:
    eleven.append(mathGrp.iat[i,0])
    i = i + 4
i = 2
while i < 60:
    twelve.append(mathGrp.iat[i,0])
    i = i + 4
# creates a final dataframe and adds our lists as columns    
mathScores = math.loc[:, ['school_name', 'math_score']]
mathScores = mathScores.rename(columns={'school_name': 'School Name'})
mathScores = mathScores.groupby('School Name')
mathScores = mathScores.mean()
mathScores['9th'] = nine
mathScores['10th'] = ten
mathScores['11th'] = eleven
mathScores['12th'] = twelve
del mathScores['math_score']
mathScores['9th'] = mathScores['9th'].map("{:.2f}".format)
mathScores['10th'] = mathScores['10th'].map("{:.2f}".format)
mathScores['11th'] = mathScores['11th'].map("{:.2f}".format)
mathScores['12th'] = mathScores['12th'].map("{:.2f}".format)

mathScores

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [211]:
# creates dataframe and groups by School Name and grade and finds the average score for each grade in each school
reading = school_data_complete.loc[:, ['school_name', 'grade', 'reading_score']]
readGrp = reading.groupby(['school_name', 'grade'])
readGrp = readGrp.mean()
# loops through grouped dataframe and creates lists of each grades averages
nine = []
ten = []
eleven = []
twelve = []
i = 3
while i < 60:
    nine.append(readGrp.iat[i,0])
    i = i + 4
i = 0
while i < 60:
    ten.append(readGrp.iat[i,0])
    i = i + 4
i = 1
while i < 60:
    eleven.append(readGrp.iat[i,0])
    i = i + 4
i = 2
while i < 60:
    twelve.append(readGrp.iat[i,0])
    i = i + 4
# creates a final dataframe and adds our lists as columns    
readScores = reading.loc[:, ['school_name', 'reading_score']]
readScores = readScores.rename(columns={'school_name': 'School Name'})
readScores = readScores.groupby('School Name')
readScores = readScores.mean()
readScores['9th'] = nine
readScores['10th'] = ten
readScores['11th'] = eleven
readScores['12th'] = twelve
del readScores['reading_score']
readScores['9th'] = readScores['9th'].map("{:.2f}".format)
readScores['10th'] = readScores['10th'].map("{:.2f}".format)
readScores['11th'] = readScores['11th'].map("{:.2f}".format)
readScores['12th'] = readScores['12th'].map("{:.2f}".format)

readScores

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


In [219]:
# creates and seperates Per Student Budget into bins and displays avg scores and passing rates for each bin
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
spendBins = unformSch.loc[:, ["Per Student Budget", 'Average Math Score', 'Average Reading Score', '% Passing Math', 
                          '% Passing Reading', '% Overall Passing Rate']]
spendBins["Per Student Budget"] = pd.cut(spendBins["Per Student Budget"], spending_bins, labels=group_names)
spendBins = spendBins.groupby("Per Student Budget")
spendBins = spendBins.mean()
spendBins['Average Math Score'] = spendBins['Average Math Score'].map("{:.2f}".format)
spendBins['Average Reading Score'] = spendBins['Average Reading Score'].map("{:.2f}".format)
spendBins['% Passing Math'] = spendBins['% Passing Math'].map("{:.2f}%".format)
spendBins['% Passing Reading'] = spendBins['% Passing Reading'].map("{:.2f}%".format)
spendBins['% Overall Passing Rate'] = spendBins['% Overall Passing Rate'].map("{:.2f}%".format)
spendBins

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Per Student Budget,,,,,
<$585,83.46,83.93,93.46%,96.61%,95.04%
$585-615,83.60,83.89,94.23%,95.90%,95.07%
$615-645,79.08,81.89,75.67%,86.11%,80.89%
$645-675,77.00,81.03,66.16%,81.13%,73.65%


In [220]:
# creates and seperates Total Students into bins and displays avg scores and passing rates for each bin
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

sizeBins = unformSch.loc[:, ['Total Students', 'Average Math Score', 'Average Reading Score', '% Passing Math', 
                          '% Passing Reading', '% Overall Passing Rate']]
sizeBins['Total Students'] = pd.cut(sizeBins['Total Students'], size_bins, labels=group_names)
sizeBins = sizeBins.groupby('Total Students')
sizeBins = sizeBins.mean()
sizeBins['Average Math Score'] = sizeBins['Average Math Score'].map("{:.2f}".format)
sizeBins['Average Reading Score'] = sizeBins['Average Reading Score'].map("{:.2f}".format)
sizeBins['% Passing Math'] = sizeBins['% Passing Math'].map("{:.2f}%".format)
sizeBins['% Passing Reading'] = sizeBins['% Passing Reading'].map("{:.2f}%".format)
sizeBins['% Overall Passing Rate'] = sizeBins['% Overall Passing Rate'].map("{:.2f}%".format)
sizeBins

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Total Students,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,94.82%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,95.20%
Large (2000-5000),77.75,81.34,69.96%,82.77%,76.36%


In [224]:
# creates a dataframe grouped by school type with the avg scores and passing rates
typeBins = unformSch.loc[:, ['School Type', 'Average Math Score', 'Average Reading Score', '% Passing Math', 
                          '% Passing Reading', '% Overall Passing Rate']]
typeBins = typeBins.groupby('School Type')
typeBins = typeBins.mean()
typeBins['Average Math Score'] = typeBins['Average Math Score'].map("{:.2f}".format)
typeBins['Average Reading Score'] = typeBins['Average Reading Score'].map("{:.2f}".format)
typeBins['% Passing Math'] = typeBins['% Passing Math'].map("{:.2f}%".format)
typeBins['% Passing Reading'] = typeBins['% Passing Reading'].map("{:.2f}%".format)
typeBins['% Overall Passing Rate'] = typeBins['% Overall Passing Rate'].map("{:.2f}%".format)
typeBins

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,95.10%
District,76.96,80.97,66.55%,80.80%,73.67%
